## Übungsblatt 1

#### Packages
Hallo und herzlich willkommen zum Kommentarbereich des ersten Übungsblattes ^~^

Zuerst laden wir die ganzen [packages](https://en.wikipedia.org/wiki/Python_Package_Index).

Wir brauchen:
1. [**os**](https://docs.python.org/3/library/os.html), um die Pfade zu definieren und um neue Dateien/Ordner zu erstellen
2. [**csv**](https://docs.python.org/3/library/csv.html), um die CSV Dateien auszulesen; man könnte eine ähnliche Funktionalität mithilfe von Pandas erreichen, jedoch wollte ich ohne große Hilfsmittel die Logik nachahmen.
3. [**re**](https://docs.python.org/3/library/re.html), um mit RegEx die Spaltennamen zu finden, die die Noten enthalten.
4. [**numpy**](https://pypi.org/project/numpy/), unter dem Namen np, um den Durchschnitt und den maximalen Wert zu berechnen.

In [1]:
import os
import csv
import re

import numpy as np

#### Directories
Jetzt laden wir die ganzen Pfade für die Dateien. BASE_DIR löst mit os in unserem Arbeitsverzeichnis auf und CSV_DIR zeigt auf einen untergeordneten Ordner, der die CSVs und ihren Output speichern soll.

In [2]:
BASE_DIR: str = os.getcwd()
CSV_DIR: str = os.path.join(BASE_DIR, 'csv')

#### Dateneingang
Diese Funktion liest die CSV Datei Reihe für Reihe aus. Sollte der CSV Ordner nicht existieren so kann man dieses Programm ohne Argumente ausführen, damit einer initialisiert wird.

Es wird überprüft, ob unser Pfadzeiger tatsächlich auf eine CSV Datei hinweist und ob diese existiert. Sollte dies zutreffen wird die Datei Zeile für Zeile ausgelesen und in einer Liste, die Listen enthält gespeichert, quasi wie ein 3D-Array. Sonst erhebt die Funktion eine Exception, die gefangen werden muss.

In [3]:
def read_csv_data(file_name: str) -> list:
    csv_data: list = []

    if not os.path.exists(CSV_DIR):
        os.makedirs(CSV_DIR)

    if file_name.endswith('.csv') and os.path.exists(file_name):
        with open(file=file_name, mode='r') as csv_file:
            csv_in = csv.reader(csv_file, quotechar='"', delimiter=";")
            for row in csv_in:
                csv_data.append(row)
    else:
        raise Exception("Path given doesn't point to a .csv file")

    return csv_data

#### Datenanalyse
Diese Funktion analysiert die Daten pro Reihe, berechnet Max und Mean und schreibt sie in eine neue Liste, die zurückgegeben wird.

Bei der ersten Zeile, die die Überschriften enthält, fügen wir die Spaltennamen 'G_MAX' und 'G_MEAN' hinzu, wobei das erstere die beste erhaltene Note ist und das letztere der Notendurchschnitt.

Danke RegEx kann sich die Anzahl der Spalten oder deren Namen änder und wir können mit minimalem Aufwand diese Änderungen umsetzen. Wir speichern die Indizes der Notenfelder, um sie später auswerten.

Zum Auswerten benutzen wir numpy, der die nötigen Operationen integriert, somit lässt sich unser Problem durch Einzeiler lösen. Wir runden die Durchschnittswerte ab, damit diese Excel nicht kaputt machen, Gott weißt dass sie es können. ☺

In [4]:
def analyze_student_results(csv_data: list) -> list:
    csv_processed: list = []
    g_columns: list = []
        
    for idx, row in enumerate(csv_data):
        if idx == 0:
            for g_idx, cell in enumerate(row):
                if re.match(r"^G[0-9]$", cell):
                    g_columns.append(g_idx)

            csv_processed.append(row + ['G_MAX', 'G_MEAN'])

        else:
            grades: list = []
                
            for g_column in g_columns:
                grades.append(int(row[g_column]))

            g_max: int = np.max(grades)
            g_mean: float = round(np.average(grades), 2)
            csv_processed.append(row + [str(g_max), str(g_mean)])
            
    return csv_processed

#### Datenausgang
Diese Funktion schreibt die Dateien in eine Datei zurück. Es wird ein Pfad und die Inhalte der zu erstellenden CSV gegeben, wobei zu beachten ist, dass Namenskonflikte im Überschreiben der existierenden Datei enden.

Wiederholt wird mittels des csv Packages der Datenfluss in die Datei gedumpt.

In [5]:
def write_csv_data(csv_data: list, file_name: str) -> None:
    if os.path.exists(file_name):
        os.remove(file_name)

    with open(file=file_name, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, quotechar='"', delimiter=";")
        for row in csv_data:
            csv_writer.writerow(row)

#### Testlauf
Kurze Testausführung unserer Logik. Wir laden zuerst die Datei 'student-mat.csv', die die Noten der Schüler und deren Noten enthält. 

Danach schicken wir sie zur Analyse, damit der Durchschnitt und der Maximalwert hinzugefügt werden. Das Ergebnis wird in csv_data_processed gespeichert.

Anschließend schicken wir die Inhalte unserer Listen zurück in eine CSV-Datei.

student_amount zählt im Einzeiler die Menge der Reihen und subtrahiert die erste, da diese nur Spaltennamen enthält.

In [6]:
file_in = os.path.join(CSV_DIR, 'student-mat.csv')
student_amount = sum(1 for line in open(file_in)) - 1

file_out = os.path.join(CSV_DIR, f'student-mat-out-total-{student_amount}.csv')

csv_data = read_csv_data(file_name=file_in)
csv_data_processed = analyze_student_results(csv_data=csv_data)
write_csv_data(csv_data=csv_data_processed, file_name=file_out)

Created 2023 by [2Batou4U](https://github.com/2Batou4U)